<a href="https://www.kaggle.com/code/irwanprabowo/healthy-lunch-planner-ai?scriptVersionId=281096752" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# ======================================================
#  MULTI-AGENT HEALTHY LUNCH PLANNER AI (REFINED VERSION)
#  Gemini 2.0 Flash + Tools + Memory + CSV (30-item dataset)
# ======================================================

import json, random, os
import pandas as pd

# ======================================================
#  GEMINI 2.0 SETUP  (new google.genai)
# ======================================================
USE_GEMINI = False
try:
    from google import genai
    GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY", "").strip()
    
    if GEMINI_API_KEY:
        client = genai.Client(api_key=GEMINI_API_KEY)
        USE_GEMINI = True
        
except Exception as e:
    print("Gemini unavailable:", e)
    USE_GEMINI = False


def call_gemini(prompt):
    """Unified Gemini wrapper with graceful fallback."""
    if USE_GEMINI:
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt
            )
            return response.text
        except Exception as e:
            return f"(Gemini error: {e})"
    
    # Offline fallback
    return "(Offline) Basic plan: filter meals by calories, price, and generate ranking."


# ======================================================
#  MEMORY SYSTEM
# ======================================================
class InMemorySessionService:
    def __init__(self):
        self.sessions = {}

    def get(self, sid):
        return self.sessions.setdefault(sid, {})

    def set(self, sid, data):
        self.sessions[sid] = data


class MemoryBank:
    def __init__(self, path="lunch_memory.json"):
        self.path = path

    def load(self):
        return json.load(open(self.path)) if os.path.exists(self.path) else {}

    def save(self, data):
        json.dump(data, open(self.path, "w"), indent=2)


# ======================================================
#  TOOLS
# ======================================================
def csv_tool():
    """
    Load the expanded 30-item meals dataset.
    Update folder name after uploading to Kaggle.
    """
    try:
        path = "/kaggle/input/meals-large-30/meals_large_30.csv"  # <-- UPDATE IF NECESSARY
        df = pd.read_csv(path)
        return df.to_dict(orient="records")
    except Exception as e:
        print("CSV loading failed:", e)
        print("Using fallback small dataset.")
        return [
            {"meal": "Chicken Salad", "calories": 450, "price": 7.5},
            {"meal": "Veggie Wrap", "calories": 380, "price": 6.0},
            {"meal": "Sushi Bento", "calories": 520, "price": 9.0},
            {"meal": "Beef Bowl", "calories": 640, "price": 8.5},
        ]


def rating_api(meal):
    """Mock meal rating API [3.5 - 5.0]."""
    return round(random.uniform(3.5, 5.0), 1)


def code_tool_compute(meals):
    """Compute averages, scoring, and ranking."""
    avg_price = sum(m["price"] for m in meals) / len(meals)
    avg_cal = sum(m["calories"] for m in meals) / len(meals)

    # Score (lower is better): (calories / 10) + price
    for m in meals:
        m["score"] = round((m["calories"] / 10) + m["price"], 2)

    ranked = sorted(meals, key=lambda x: x["score"])

    return {
        "avg_price": round(avg_price, 2),
        "avg_calories": round(avg_cal, 2),
        "ranked_meals": ranked[:5],     # top 5 recommendations
        "best_meal": ranked[0]          # single best candidate
    }


# ======================================================
#  AGENTS
# ======================================================
class Planner:
    def run(self, goal):
        prompt = f"""
        You are the Planner Agent in a Multi-Agent AI.

        User goal:
        - Budget: ${goal['budget']}
        - Max calories: {goal['calories']}

        Produce a clear JSON plan with:
        - objective
        - 4–6 steps
        - which agent handles each step
        - success criteria
        """
        return call_gemini(prompt)


class Researcher:
    def run(self):
        meals = csv_tool()
        enriched = []

        for m in meals:
            m["rating"] = rating_api(m["meal"])

            # Add insights via Gemini
            prompt = f"""
            Give one short nutritional insight for this meal:

            Meal: {m['meal']}
            Calories: {m['calories']}
            Protein: {m['protein']}
            Fat: {m['fat']}
            Carbs: {m['carbs']}
            Rating: {m['rating']}
            """
            m["insight"] = call_gemini(prompt)
            enriched.append(m)

        return enriched


class Coder:
    def run(self, meals):
        return code_tool_compute(meals)


class Critic:
    def run(self, summary, goal):
        prompt = f"""
        You are the Critic Agent.

        Evaluate the following data:

        Summary: {summary}
        User goal: {goal}

        Respond in JSON:
        - judgment: "good" or "needs_improvement"
        - reason
        - recommendation
        """
        return call_gemini(prompt)


# ======================================================
#  ORCHESTRATOR
# ======================================================
class Orchestrator:
    def __init__(self):
        self.session = InMemorySessionService()
        self.memory = MemoryBank()
        self.state = self.memory.load() or {}

    def run(self, goal):
        print("=== HEALTHY LUNCH PLANNER AI (30-Item Version) ===")

        planner = Planner()
        researcher = Researcher()
        coder = Coder()
        critic = Critic()

        print("\n📝 Planner Agent Output:")
        plan = planner.run(goal)
        print(plan)

        print("\n🔍 Researcher Agent Output:")
        meals = researcher.run()
        for m in meals[:5]:
            print(m)

        print("\n💻 Coder Agent Summary:")
        summary = coder.run(meals)
        print(summary)

        print("\n🧩 Critic Agent Evaluation:")
        evaluation = critic.run(summary, goal)
        print(evaluation)

        # Save memory state
        self.state = {
            "goal": goal,
            "planner": plan,
            "meals": meals,
            "summary": summary,
            "evaluation": evaluation
        }
        self.memory.save(self.state)

        print("\n💾 State saved to lunch_memory.json")


# ======================================================
#  RUN AGENT SYSTEM
# ======================================================
goal = {"budget": 8.0, "calories": 600}
Orchestrator().run(goal)


=== HEALTHY LUNCH PLANNER AI (30-Item Version) ===

📝 Planner Agent Output:
(Offline) Basic plan: filter meals by calories, price, and generate ranking.

🔍 Researcher Agent Output:
{'meal': 'Chicken Salad', 'calories': 450, 'price': 7.5, 'protein': 32, 'fat': 12, 'carbs': 20, 'rating': 4.8, 'insight': '(Offline) Basic plan: filter meals by calories, price, and generate ranking.'}
{'meal': 'Veggie Wrap', 'calories': 380, 'price': 6.0, 'protein': 10, 'fat': 8, 'carbs': 55, 'rating': 4.7, 'insight': '(Offline) Basic plan: filter meals by calories, price, and generate ranking.'}
{'meal': 'Sushi Bento', 'calories': 520, 'price': 9.0, 'protein': 28, 'fat': 14, 'carbs': 60, 'rating': 3.7, 'insight': '(Offline) Basic plan: filter meals by calories, price, and generate ranking.'}
{'meal': 'Beef Bowl', 'calories': 640, 'price': 8.5, 'protein': 35, 'fat': 20, 'carbs': 65, 'rating': 4.8, 'insight': '(Offline) Basic plan: filter meals by calories, price, and generate ranking.'}
{'meal': 'Tofu Stir 